In [51]:
from IPython.core.display import display, HTML
from string import Template
import json, random

In [52]:
HTML('''
<script src="lib/sigmajs/sigma.min.js"></script>
<script src="js/sigma-add-method-neighbors.js"></script>
''')

In [53]:
%%script bash
arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  

#CONSTRUCT 
#{
#    ?x rdfs:subClassOf ?o .
#}
SELECT ?x ?xlabel ?o ?olabel
WHERE
{
    ?x rdfs:subClassOf ?o .
    filter(!isblank(?o)) 
    filter(!regex(str(?o), "PATO"))
    filter(regex(str(?o), "TO"))
    ?x rdfs:label ?xlabel . 
    ?o rdfs:label ?olabel.
}' > toSigma.json

In [85]:
with open('to.json') as data_file:    
    data = json.load(data_file)
    list = data["results"]["bindings"]
    
    graph = {"nodes": [], "edges": []}
    j = 0
    
    xcord = random.uniform(0,1)
    ycord = random.uniform(0,1)
    for entry in list:
        x = 0
        o = 0
        
        for e in graph["nodes"]:
            if e["id"] == entry["x"]["value"]:
                x=1
            if e["id"] == entry["o"]["value"]:
                o=1
        if x==0:
            #graph["nodes"].append( {"name": entry["xlabel"]["value"], "group": 1} )
             graph['nodes'].append({
            "id": entry["x"]["value"],
          #  "label": entry["xlabel"]["value"],
            "x": random.uniform(0,1),
            "y": random.uniform(0,1),
            "size": 1
        })
             xcord+=1
             ycord+=1
                
        if o==0:
         graph['nodes'].append({
            "id": entry["o"]["value"],
           # "label": entry["olabel"]["value"],
            "x": random.uniform(0,1),
            "y": random.uniform(0,1),
            "size": 1
        })
         xcord+=1
         ycord+=1
            
        if  entry["x"]["value"] != entry["o"]["value"]:
            j +=1
            graph['edges'].append({
                "id": "e" + str(j),
                "source": entry["x"]["value"],
                "target": entry["o"]["value"], 
                "size": 100
            })
           

In [86]:
graph

{'edges': [{'id': 'e1',
   'size': 100,
   'source': 'http://purl.obolibrary.org/obo/TO_0006045',
   'target': 'http://purl.obolibrary.org/obo/TO_0001024'},
  {'id': 'e2',
   'size': 100,
   'source': 'http://purl.obolibrary.org/obo/TO_0002666',
   'target': 'http://purl.obolibrary.org/obo/TO_0001024'},
  {'id': 'e3',
   'size': 100,
   'source': 'http://purl.obolibrary.org/obo/TO_0000645',
   'target': 'http://purl.obolibrary.org/obo/TO_0001024'},
  {'id': 'e4',
   'size': 100,
   'source': 'http://purl.obolibrary.org/obo/TO_0000626',
   'target': 'http://purl.obolibrary.org/obo/TO_0001024'},
  {'id': 'e5',
   'size': 100,
   'source': 'http://purl.obolibrary.org/obo/TO_0020114',
   'target': 'http://purl.obolibrary.org/obo/TO_0001024'},
  {'id': 'e6',
   'size': 100,
   'source': 'http://purl.obolibrary.org/obo/TO_0001025',
   'target': 'http://purl.obolibrary.org/obo/TO_0001024'},
  {'id': 'e7',
   'size': 100,
   'source': 'http://purl.obolibrary.org/obo/TO_0006019',
   'target': '

In [87]:
js_text_template = Template(open('js/sigma-graph.js','r').read())

js_text = js_text_template.substitute({'graph_data': json.dumps(graph),
                                       'container': 'graph-div'})

html_template = Template('''
<div id="graph-div" style="height:400px"></div>
<script> $js_text </script>
''')
HTML(html_template.substitute({'js_text': js_text}))